In [ ]:
$workingFolder = '.'
$kernelPath = $((jupyter kernelspec list | Select-String 'python3' | ForEach-Object { ($_ -replace '^\s+', '') -split '\s+' })[1] | Split-Path)
Write-Host "Install to $kernelPath"

In [ ]:
$ProgressPreference = 'SilentlyContinue'
$links = (Invoke-WebRequest -Uri 'https://dotnet.microsoft.com/download' -UseBasicParsing).Links.href
$latestVer = (($links | Select-String -Pattern '.*sdk.*windows-x64-installer') -replace '.*sdk-(([0-9]+\.){1}[0-9]+(\.[0-9]+)?)-.*', '$1' | Measure-Object -Maximum).Maximum
$latestUri = 'https://dotnet.microsoft.com' + ($links | Select-String -Pattern ".*sdk-$latestVer-windows-x64-installer" | Get-Unique).Tostring().Trim()
$fileUri = ((Invoke-WebRequest -Uri $latestUri -UseBasicParsing).Links.href | Select-String -Pattern '.*\.exe' | Get-Unique).Tostring().Trim()
Invoke-WebRequest -Uri $fileUri -UseBasicParsing -OutFile (Join-Path $workingFolder 'dotnet.exe') -Verbose

In [ ]:
Start-Process -FilePath (Join-Path $WorkingFolder 'dotnet.exe') -ArgumentList '/install /passive /norestart' -Wait

In [ ]:
Remove-Item (Join-Path $WorkingFolder 'dotnet.exe') -Force -Verbose

In [ ]:
$env:Path = [System.Environment]::GetEnvironmentVariable("Path","Machine") + ";" + [System.Environment]::GetEnvironmentVariable("Path","User")
dotnet tool install -g --add-source "https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json" Microsoft.dotnet-interactive
dotnet interactive jupyter install --path "$kernelPath"